In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp -r -v drive/My\ Drive/ThaiLanguageModel/* .
!rsync -a --progress --exclude="dataset" drive/My\ Drive/ThaiLanguageModel/* .

sending incremental file list
README.md
             19 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=5/6)
character_copus.txt
            259 100%    0.72kB/s    0:00:00 (xfr#2, to-chk=4/6)
data_gen.py
          2,559 100%    3.13kB/s    0:00:00 (xfr#3, to-chk=3/6)
model.ipynb
         74,896 100%   58.09kB/s    0:00:01 (xfr#4, to-chk=2/6)
model.py
              0 100%    0.00kB/s    0:00:00 (xfr#5, to-chk=1/6)
wiki_prepare.py
          2,019 100%    4.59kB/s    0:00:00 (xfr#6, to-chk=0/6)


In [0]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/thwiki-words
!cp drive/My\ Drive/ThaiLanguageModel/dataset/thwiki-words.zip dataset/thwiki-words.zip
!unzip dataset/thwiki-words.zip -d dataset

In [1]:
!pip install ujson
!pip install pythainlp

     |████████████████████████████████| 194kB 1.1MB/s eta 0:00:01
  Created wheel for ujson: filename=ujson-1.35-cp37-cp37m-linux_x86_64.whl size=69600 sha256=effde6ec631d01ff6dc88b2e273b4d1064dae9e3b031532d02f2fc16ff7f6144
  Stored in directory: /home/porlolicon/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
     |████████████████████████████████| 11.0MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 10.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/20/95/d23071d0992dabcb61c948fb118a90683193befc88c23e745b050a29e7db/marisa-trie-0.7.5.tar.gz
     |████████████████████████████████| 61kB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 10.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 10.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4b

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from keras.layers import CuDNNLSTM, Bidirectional, RepeatVector, Dense, TimeDistributed, Activation, Input
from keras.models import Sequential
from keras.optimizers import Adam
from pathlib import Path
import data_gen

In [3]:
NUM_VOCAB = data_gen.NUM_VOCAB
length = 160
batch_size = 16
train_gen = data_gen.sentence_generator(Path('dataset/thwiki-words'),
                                        length,
                                        batch_size,
                                        adversarial=data_gen.random_blank(0.00))

In [12]:
model = Sequential()
activation = 'relu'
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True), input_shape=(length, NUM_VOCAB)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(32, )))
model.add(Activation(activation))
model.add(RepeatVector(length))
model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Activation(activation))
model.add(TimeDistributed(Dense(NUM_VOCAB)))
model.add(Activation('softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_34 (Bidirectio (None, 160, 256)          221184    
_________________________________________________________________
activation_34 (Activation)   (None, 160, 256)          0         
_________________________________________________________________
bidirectional_35 (Bidirectio (None, 160, 256)          395264    
_________________________________________________________________
activation_35 (Activation)   (None, 160, 256)          0         
_________________________________________________________________
bidirectional_36 (Bidirectio (None, 160, 128)          164864    
_________________________________________________________________
activation_36 (Activation)   (None, 160, 128)          0         
_________________________________________________________________
bidirectional_37 (Bidirectio (None, 160, 128)         

In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch=4096,
                    epochs=100)

Epoch 1/100
4096/4096 [==============================] - 613s 150ms/step - loss: 3.6205 - acc: 0.0989
Epoch 2/100
4096/4096 [==============================] - 611s 149ms/step - loss: 3.6106 - acc: 0.0996
Epoch 3/100
4096/4096 [==============================] - 612s 149ms/step - loss: 3.6064 - acc: 0.1003
Epoch 4/100
3459/4096 [========================>.....] - ETA: 1:34 - loss: 3.6098 - acc: 0.0989